<a href="https://colab.research.google.com/github/takusandesu/kaggle/blob/main/atema11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os

import pandas as pd
import numpy as np
from glob import  glob

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import torch
from torch import nn
from torch.optim import Adam
from torch.optim.optimizer import Optimizer
from torch.utils import data

# torchvision
from torchvision import transforms as T
from torchvision.models import resnet34

# scikit-learn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [5]:
!pip install python-vivid

     |████████████████████████████████| 76 kB 2.4 MB/s 
     |████████████████████████████████| 302 kB 9.6 MB/s 
     |████████████████████████████████| 208 kB 37.4 MB/s 
     |████████████████████████████████| 80 kB 8.3 MB/s 
     |████████████████████████████████| 75 kB 4.2 MB/s 
     |████████████████████████████████| 111 kB 49.1 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
     |████████████████████████████████| 144 kB 49.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=7f0ed8aa7dc366a450688a24d65facce726b8792aa0365616968f2663ecbef69
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


### データの読み込みと画像ディレクトリのパス

In [31]:
train_df = pd.read_csv("/content/drive/MyDrive/atmaCup11/inputs/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/atmaCup11/inputs/test.csv")

dataset_root = '/content/drive/MyDrive/atmaCup11/'

input_dir = os.path.join(dataset_root, "inputs")
photo_dir = os.path.join(input_dir, "photos")

output_dir = os.path.join(dataset_root, "outputs_tutorial#1__simple")
os.makedirs(output_dir, exist_ok=True)

In [32]:
photo_dir

'/content/drive/MyDrive/atmaCup11/inputs/photos'

In [34]:
glob('/content/drive/MyDrive/atmaCup11/inputs/*')

['/content/drive/MyDrive/atmaCup11/inputs/techniques.csv',
 '/content/drive/MyDrive/atmaCup11/inputs/test.csv',
 '/content/drive/MyDrive/atmaCup11/inputs/train.csv',
 '/content/drive/MyDrive/atmaCup11/inputs/atmaCup#11_sample_submission.csv',
 '/content/drive/MyDrive/atmaCup11/inputs/materials.csv',
 '/content/drive/MyDrive/atmaCup11/inputs/photos.zip',
 '/content/drive/MyDrive/atmaCup11/inputs/photos',
 '/content/drive/MyDrive/atmaCup11/inputs/data.zip']

In [35]:
train_df.head()

,object_id,sorting_date,art_series_id,target
0,002bff09b09998d0be65,1631,509357f67692a6a45626,1
1,00309fb1ef05416f9c1f,1900,7987b47bbe5dc3039179,3
2,003a1562e97f79ba96dc,1834,ded7c3c9636708e5b14c,3
3,004890880e8e7431147b,1743,582ac2d7f0cef195b605,2
4,00718c32602425f504c1,1885,64c907f0c08dce4fb8e8,3


### バリデーションの回数とエポック数

In [36]:
N_FOLDS = 5
N_EPOCHS = 1

### object_idから画像のパスを返す関数と画像を開く関数(to_img_path,read_image)

- `to_img_path`でobject_idを渡して、画像のパスを返す
- object_idを渡して、画像を開く

In [25]:
from PIL import Image

def to_img_path(object_id):
    return os.path.join(photo_dir, f'{object_id}.jpg')#photoの名前がobject_id.jpgになっている
    #画像の相対パス
def read_image(object_id):
    return Image.open(to_img_path(object_id))#画像のパス

### 処理の流れ
1. trainデータのobject_idを画像へのパスに変換してtrainデータ(object_id,object_patah,target)を作る
  - create_metadata(input_df)
2. trainデータをtrain(学習用)とval(検証用)にk-foldで分割
3. 使用するモデルを定義する(resnet34)
  - create_model()
4. trainとvalをミニバッチ化してtrainで学習し、valで検証する
   - run_fold(
    model: nn.Module, 
    train_df: pd.DataFrame, 
    valid_df: pd.DataFrame, 
    y_valid: np.ndarray, 
    n_epochs=30) -> np.ndarray:
    - train(
    model: nn.Module,
    optimizer: Optimizer,
    train_loader: data.DataLoader
) -> pd.Series:
    - valid(
    model: nn.Module, 
    y_valid: np.ndarray, 
    valid_loader: data.DataLoader
) -> pd.Series:
      -  predict(model: nn.Module, loader: data.DataLoader) -> np.ndarray:
      - calculate_metrics(y_true, y_pred)
  
5. testデータで推論する
  -  run_test_predict(model)
    - predict(model: nn.Module, loader: data.DataLoader) -> np.ndarray:

### データセットを読み込むclass(AtmaDataset)
- 1枚ずつ読み込み処理をするのは大変
- `torch.utils.data.DataSet`の活用(継承)
- `to_dict(orient="index")`で行をキーとして値をvalueにした辞書
-  [to_dictについて](https://note.nkmk.me/python-pandas-to-dict/)

- `__getitem__`ではindexのintを引数にとって、その時のデータを返す
- 画像へのパスとラベルを暗記

In [75]:
from torch.utils import data

IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]

class AtmaDataset(data.Dataset):
    """atmaCup用にデータ読み込み等を行なうデータ・セット"""
    object_path_key = "object_path"
    label_key = "target"

    @property
    def meta_keys(self):
        retval = [self.object_path_key]

        if self.is_train:
            retval += [self.label_key]

        return retval

    def __init__(self, meta_df: pd.DataFrame, is_train=True):
        """
        args:
            meta_df: 
                画像へのパスと label 情報が含まれている dataframe
                必ず object_path に画像へのパス, target に正解ラベルが入っている必要があります

            is_train:
                True のとき学習用のデータ拡張を適用します.
                False の時は単に size にリサイズを行います
        """

        self.is_train = is_train
        for k in self.meta_keys:
            if k not in meta_df:
                raise ValueError("meta df must have {}".format(k))

        self.meta_df = meta_df.reset_index(drop=True)
        self.index_to_data = self.meta_df.to_dict(orient="index")

        size = (224, 224)

        additional_items = (
            [T.Resize(size)]
            if not is_train
            else [
                T.RandomGrayscale(p=0.2),
                #ランダムに上下反転を行う
                T.RandomVerticalFlip(),
                T.RandomHorizontalFlip(),
                #ランダムに明るさ、コントラスト、彩度、色相を変化させる
                T.ColorJitter(
                    brightness=0.3,
                    contrast=0.5,
                    saturation=[0.8, 1.3],
                    hue=[-0.05, 0.05],
                ),
                #ランダムにリサイズ及び切り抜きを行う
                T.RandomResizedCrop(size),
            ]
        )

        self.transformer = T.Compose(
            [*additional_items, T.ToTensor(), T.Normalize(mean=IMG_MEAN, std=IMG_STD)]
        )

    def __getitem__(self, index):
				#index_to_dataは辞書型で{行数:{"target":value,"object_id":value,"object_path":value}}
        data = self.index_to_data[index]
				#.get(-1)では存在するならvaluesを返し、ないなら-1を返す
        obj_path, label = data.get(self.object_path_key), data.get(self.label_key, -1)
        img = Image.open(obj_path)
        img = self.transformer(img)
        return img, label

    def __len__(self):
        return len(self.meta_df)

In [76]:
train_meta_df = train_df[['target', 'object_id']].copy()
#to_img_pathはobjectidから画像のpathを出力できる関数で、それを列全てに実行
train_meta_df['object_path'] = train_meta_df['object_id'].map(to_img_path)

dataset = AtmaDataset(meta_df=train_meta_df)

In [77]:
loader = data.DataLoader(dataset=dataset, batch_size=54, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [78]:
#x_tensorとyはともにtensor型
for x_tensor, y in loader:
    break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


バッチサイズが54のミニパッチ化され、imgとlabelがミニバッチが一塊としてloaderに入っている

In [79]:
x_tensor.shape, y.shape

(torch.Size([54, 3, 224, 224]), torch.Size([54]))

### GPU環境に合わせる
- GPUの使用ではモデル、変数、計算に関わるものすべてを .to(device) しなければならない

In [41]:
assert torch.cuda.is_available()
DEVICE = torch.device("cuda")

### 学習の関数 : train

In [45]:
def train(
    model: nn.Module,
    optimizer: Optimizer,
    train_loader: data.DataLoader
) -> pd.Series:

    # train にすることでmodel内の学習時にのみ有効な機構が有効になります 
    model.train()#学習

    criterion = nn.MSELoss()#評価関数

    for i, (x_i, y_i) in enumerate(train_loader):
        x_i = x_i.to(DEVICE)
        y_i = y_i.to(DEVICE).reshape(-1, 1).float()

        output = model(x_i)#モデルに学習データを入れる
        loss = criterion(output, y_i)#学習した値と目的変数のlossを計算　lossはtensorオブジェクト


        optimizer.zero_grad()#勾配の初期化
        loss.backward()#勾配計算
        optimizer.step()#パラメータの微小移動

In [95]:
inputs = torch.randn(10, 32)
targets = torch.randn(10)
targets = targets.view(1, -1)
targets.float()
np.array(y.numpy()).reshape(-1)
np.ones((3,4)).reshape(-1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

### 予測の関数 : predict
- [Pytorch tensor と numpy ndarray の変換 ](https://tzmi.hatenablog.com/entry/2020/02/16/170928)
- 予測値の配列をndarrayで返す

In [46]:
def predict(model: nn.Module, loader: data.DataLoader) -> np.ndarray:
    # train とは逆で model 内の学習時にのみ有効な機構がオフになります (Dropouts Layers、BatchNorm Layers...)
    model.eval()#推測

    predicts = []

    for x_i, y_i in loader:

        # 明示的に勾配を計算しないように指定することができます. 
        # この関数ではモデルの更新はせずに単に出力だけを使いますので勾配は不要です.
        with torch.no_grad():#勾配を計算しない
            output = model(x_i.to(DEVICE))#モデルに入れて推論

        #リストの末尾に別のリストやタプルを結合
        predicts.extend(output.data.cpu().numpy())#Tensor型からarrayに

    pred = np.array(predicts).reshape(-1)#1列になる
    return pred

### 予測値の評価指標をする関数(caluculate_metrics)

In [47]:
def calculate_metrics(y_true, y_pred) -> dict:
    """正解ラベルと予測ラベルから指標を計算する"""    
    return {
        'rmse': mean_squared_error(y_true, y_pred) ** .5
    }


### 検証データを推論して、評価指標を計算する(valid)

In [48]:
def valid(
    model: nn.Module, 
    y_valid: np.ndarray, 
    valid_loader: data.DataLoader
) -> pd.Series:
    """検証フェーズ
    与えられたモデル・データローダを使って検証フェーズを実行。スコアの dict と予測した値を返す
    """
		#predictのデータ
    pred = predict(model, valid_loader)
		#rmseを求める
    score = calculate_metrics(y_valid, pred)#正解ラベルと予測ラベルから指標を計算
    return score, pred

### 学習データから学習して検証データに対する評価指標を計算する関数(run_fold)

In [49]:
def run_fold(
    model: nn.Module, 
    train_df: pd.DataFrame, 
    valid_df: pd.DataFrame, 
    y_valid: np.ndarray, 
    n_epochs=30) -> np.ndarray:
    """
    train / valid に分割されたデータで学習と同時に検証を行なう
    """

    # 0: 
    #   : 前準備. dataframe から data loader を作成
		#データのidを画像へのパスに変換、かつtensor型に変換
    train_dataset = AtmaDataset(meta_df=train_df)
		#データをミニパッチ化する
    train_loader = data.DataLoader(
        train_dataset, batch_size=64, shuffle=True, drop_last=True, num_workers=4
    )

    #  : 検証用の方は is_train=False にしてデータ拡張オフにする
    valid_dataset = AtmaDataset(meta_df=valid_df, is_train=False)
    valid_loader = data.DataLoader(valid_dataset, batch_size=256, num_workers=4)

    # optimizer の定義　
    optimizer = Adam(model.parameters(), lr=1e-3)

    for epoch in range(1, n_epochs + 1):
        print(f'start {epoch}')

        # 1: 学習用データで学習を実行。学習時のロスを取得
        train(model, optimizer, train_loader)

        # 2: 検証データでのスコアを計算
        score_valid, y_valid_pred = valid(model=model, valid_loader=valid_loader, y_valid=y_valid)

        print(score_valid)

### モデル作成の関数とtrain_meta_dataを作成する関数(create_model create_metadata)

In [50]:
def create_model():
    model = resnet34(pretrained=False)#事前学習なし
    model.fc = nn.Linear(in_features=512, out_features=1, bias=True)    
    return model

def create_metadata(input_df):
    out_df = input_df[['object_id']].copy()
    out_df['object_path'] = input_df['object_id'].map(to_img_path)
		
		#input_dfのcloumnにtarget列があるなら
    if "target" in input_df:
        out_df["target"] = input_df["target"]
    return out_df

### テストデータに対して推論する関数(run_test_predict)

In [51]:
def run_test_predict(model):
    test_meta_df = create_metadata(test_df)

    # 学習時のデータ拡張はオフにしたいので is_train=False としている
    test_dataset = AtmaDataset(meta_df=test_meta_df, is_train=False)
    test_loader = data.DataLoader(dataset=test_dataset, batch_size=128, drop_last=False, num_workers=4)

    y_pred = predict(model, loader=test_loader)
    return y_pred

In [54]:
train_meta_df = create_metadata(train_df)

#テストデータの予測値を入れとく配列
test_predictions = []

fold = KFold(n_splits=5, shuffle=True, random_state=510)
#[([1,3,5],[2,4]),(),(),(),()]
cv = list(fold.split(X=train_df, y=train_df['target']))


In [59]:
cv = list(fold.split(X=train_df))
len(cv)

5

### 処理全体を表すコード

In [60]:
train_meta_df = create_metadata(train_df)

#テストデータの予測値を入れとく配列
test_predictions = []

fold = KFold(n_splits=5, shuffle=True, random_state=510)
#[([1,3,5],[2,4]),(),(),(),()]
cv = list(fold.split(X=train_df))


for i, (idx_tr, idx_valid) in enumerate(cv):
    model = create_model()
    model.to(DEVICE)

    # 1. Fold の学習
    run_fold(
        model=model, 
        train_df=train_meta_df.iloc[idx_tr], #学習用データ
        valid_df=train_meta_df.iloc[idx_valid], #検証データ
        y_valid=train_meta_df['target'].values[idx_valid],#検証データのラベル
        n_epochs=N_EPOCHS
    )
		#使いたいのは学習したモデルだけ
    # 2. モデルで予測 (本当はローカルに保存した重みを読みだすなどするほうがあとで振り返りやすいが簡易にそのまま予測する)
    y_pred_i = run_test_predict(model)#モデルごとに(今回は5個)予測値の塊を配列に追加
    test_predictions.append(y_pred_i)
    del model

start 1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


{'rmse': 0.9790975492940155}


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


start 1
{'rmse': 0.967776115561879}


...

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


start 1
{'rmse': 1.0214558382801135}


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


start 1
{'rmse': 0.997500093430277}


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### cvごとに作成したモデル(n_split)による予測値の平均を計算し結果として出力

In [73]:
pred_mean=np.average(test_predictions,axis=0)

In [74]:
pd.DataFrame({"target":pred_mean})

,target
0,1.819890
1,1.635594
2,1.606271
3,1.592294
4,1.541709
...,...
5914,1.618858
5915,1.459834
5916,1.577902
5917,1.571699
